# Author: Jonathan Johnson (@jsecurity101)

In [1]:
!pip install splunk-sdk
!pip install pandas
!pip install pandasql
import matplotlib.pyplot as plt
import pandas as pd
import pandasql
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 0)

In [4]:
import splunklib.client as client
# Create a Service instance and log in 
service = client.connect(
    host='192.168.0.0', #Input your Splunk's IP here
    port='8089',
    username="admin",
    password="Changeme1!",
    scheme='https')

In [5]:
import splunklib.results as results
#Second Query: 
query = "search index=main sourcetype=wineventlog:microsoft-windows-sysmon/operational EventCode=12 TargetObject!=*UserService* TargetObject!=*UserSvc* TargetObject!=*LanmanServer* TargetObject!=*NetLogon* TargetObject!=*Svc* TargetObject!=*UnistoreSvc* TargetObject!=*UserData* TargetObject!=*NTDS* TargetObject=*Services* Image!=*WebServices.exe Image=*services.exe | head 1000"
query_results = service.jobs.oneshot(query, count=0)
reader = results.ResultsReader(query_results)

results = []

for result in reader:
    results.append(result)
    
df_EID_12=pd.DataFrame(results)

In [6]:
import splunklib.results as results
#First Query: 
query = "search index=main sourcetype=dce_rpc-too_small  endpoint=svcctl | head 2000"
query_results = service.jobs.oneshot(query, count=0)
reader = results.ResultsReader(query_results)

results = []

for result in reader:
    results.append(result)
    
df_rpc_svcctl=pd.DataFrame(results)

In [9]:
df2=df_rpc_svcctl._raw

In [10]:
import json
dfj = [json.loads(j) for j in df2]
df2_rpc_svcctl = pd.DataFrame(dfj)

In [11]:
import splunklib.results as results
#Second Query: 
query = "search index=main index=main sourcetype=wineventlog:security EventCode=5156 (Application_Name!=*microsoft.activedirectory.webservices.exe AND Application_Name!=System AND Application_Name!=*launcher.exe AND Application_Name!=*googleupdate.exe AND Application_Name!=*backgrounddownload.exe AND Application_Name!=*splunk*.exe AND Application_Name!=*localbridge.exe AND Application_Name!=*microsoft.photos.exe AND Application_Name!=*hxtsr.exe AND Application_Name!=*svchost.exe AND Application_Name!=*lsass.exe AND Application_Name!=*dfsrs.exe AND Application_Name!=*searchui.exe AND Application_Name!=*dns.exe) | head 1000"
query_results = service.jobs.oneshot(query, count=0)
reader = results.ResultsReader(query_results)

results = []

for result in reader:
    results.append(result)
    
df_EID_5156_Client=pd.DataFrame(results)

In [28]:
import splunklib.results as results
#Second Query: 
query = "search index=main index=main sourcetype=wineventlog:security EventCode=5156 Application_Name=*services.exe | head 1000"
query_results = service.jobs.oneshot(query, count=0)
reader = results.ResultsReader(query_results)

results = []

for result in reader:
    results.append(result)
    
df_EID_5156_Server=pd.DataFrame(results)

In [52]:
df2_EID_5156_Client = df_EID_5156_Client.Message
df2_EID_5156_Server = df_EID_5156_Server.Message
df2_EID_12 = df_EID_12.Message

In [53]:
df3_EID_12 = df2_EID_12.str.split('\n')
df3_EID_5156_Client = df2_EID_5156_Client.str.split('\n')
df3_EID_5156_Server = df2_EID_5156_Server.str.split('\n')

In [54]:
df4_EID_12 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_12]
df4_EID_5156_Client = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_5156_Client]
df4_EID_5156_Server = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_5156_Server]

In [55]:
df5_EID_12 = pd.DataFrame(df4_EID_12)
df5_EID_5156_Client = pd.DataFrame(df4_EID_5156_Client)
df5_EID_5156_Server = pd.DataFrame(df4_EID_5156_Server)

In [56]:
df = df5_EID_5156_Server['Application Name'].str.split('\\').str[-1]
df1 = df5_EID_5156_Client['Application Name'].str.split('\\').str[-1]
df5_EID_5156_Server.update(df)
df5_EID_5156_Client.update(df1)

In [57]:
display(df5_EID_5156_Server)

,Process ID,Application Name,Direction,Source Address,Source Port,Destination Address,Destination Port,Protocol,Filter Run-Time ID,Layer Name,Layer Run-Time ID
0,1936,microsoft.activedirectory.webservices.exe,Outbound,::1,57196,::1,389,6,66620,Connect,50
1,1936,microsoft.activedirectory.webservices.exe,Outbound,::1,57195,::1,389,6,66620,Connect,50
2,1936,microsoft.activedirectory.webservices.exe,Outbound,::1,57183,::1,389,6,66620,Connect,50
3,1936,microsoft.activedirectory.webservices.exe,Outbound,::1,57178,::1,389,6,66620,Connect,50
4,1936,microsoft.activedirectory.webservices.exe,Outbound,::1,57177,::1,389,6,66620,Connect,50
...,...,...,...,...,...,...,...,...,...,...,...
995,2136,microsoft.activedirectory.webservices.exe,Outbound,::1,50543,::1,389,6,66620,Connect,50
996,2136,microsoft.activedirectory.webservices.exe,Outbound,::1,50539,::1,389,6,66620,Connect,50
997,2136,microsoft.activedirectory.webservices.exe,Outbound,::1,50539,::1,389,6,66620,Connect,50
998,2136,microsoft.activedirectory.webservices.exe,Outbound,::1,50537,::1,389,6,66620,Connect,50


In [64]:
from pandasql import sqldf
Event_5156_Zeek_df = pandasql.sqldf(
"""
SELECT -- This event will show Registry Events and the RPC Server Application
a."Source Address",
a."Destination Address",
a."Source Port",
a."Destination Port", 
a."Application Name", 
b."operation"
FROM df5_EID_5156_Server a
JOIN df2_rpc_svcctl b
ON b."id.orig_h" = a."Source Address"
AND a."Application Name" = "services.exe"
AND a.Direction = "Inbound"
AND (b."operation" LIKE "%CreateService%" OR b."operation" LIKE "%OpenSCManager%")
"""

)

In [65]:
display(Event_5156_Zeek_df)

,Source Address,Destination Address,Source Port,Destination Port,Application Name,operation
0,192.168.72.4,192.168.72.3,49994,49676,services.exe,CreateServiceW
1,192.168.72.4,192.168.72.3,49994,49676,services.exe,OpenSCManagerW
2,192.168.72.4,192.168.72.3,49994,49676,services.exe,OpenSCManagerW


In [253]:
from pandasql import sqldf
Event_5156_12_df = pandasql.sqldf(
"""
-- This event will show Registry Events and the RPC Server Application
SELECT 
a."Application Name", 
b."Application Name", 
a."Destination Port",
b."Source Address",
b."Destination Address", 
c.TargetObject
FROM df5_EID_5156_Client  a
JOIN df5_EID_5156_Server b
ON a."Application Name" != b."Application Name"
AND a."Destination Port" = b."Destination Port"
AND b.Direction = "Inbound"
AND a.Direction = "Outbound"
JOIN df5_EID_12 c
ON b."Process Id" = c.ProcessId

"""

)

In [254]:
display(Server_Side_df)

,Application Name,Application Name,Destination Port,Source Address,Destination Address,TargetObject
0,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
1,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
2,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
3,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
4,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
...,...,...,...,...,...,...
95,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
96,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
97,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
98,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test


In [255]:
#Client + Server Side Data
from pandasql import sqldf
Client_Server_Side_df = pandasql.sqldf(
"""
SELECT 
e.operation,
a."Application Name", 
b."Application Name", 
a."Destination Port",
b."Source Address",
b."Destination Address",  
c.TargetObject
FROM df5_EID_5156_Client  a
JOIN df5_EID_5156_Server b
ON a."Application Name" != b."Application Name"
AND a."Destination Port" = b."Destination Port"
AND b.Direction = "Inbound"
AND a.Direction = "Outbound"
JOIN df5_EID_12 c
ON b."Process Id" = c.ProcessId
JOIN df2_rpc_svcctl e
ON e."id.resp_p" = a."Destination Port"
AND e.operation LIKE "%CreateService%"

"""

)

In [256]:
display(Client_Server_Side_df)

,operation,Application Name,Application Name,Destination Port,Source Address,Destination Address,TargetObject
0,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
1,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
2,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
3,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
4,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
...,...,...,...,...,...,...,...
295,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
296,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
297,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
298,CreateServiceW,\device\harddiskvolume2\windows\system32\sc.exe,\device\harddiskvolume2\windows\system32\services.exe,49678,192.168.146.6,192.168.146.3,HKLM\System\CurrentControlSet\Services\test
